<a href="https://colab.research.google.com/github/EunSik312/lecture-deeplearning/blob/main/7%EC%9B%9418%EC%9D%BCyolo11_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://youtu.be/KtBa71KDNkQ?si=95TPlwWchtXfz87U

In [2]:
!pip install yt-dlp ultralytics opencv-python

from google.colab import files
from ultralytics import YOLO
import yt_dlp
import os
import cv2
from IPython.display import Video, display
model = YOLO("yolo11n.pt")
model.info()
results = model.train(data="coco8.yaml", epochs=10, imgsz=640)

# 유튜브 영상 다운로드 함수
def download_youtube_video(url):
    """유튜브 영상을 다운로드하는 함수"""
    output_path = "./downloads"
    os.makedirs(output_path, exist_ok=True)

    ydl_opts = {
        'format': 'best[height<=720]',  # 720p 이하 최고 화질
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            title = info.get('title', 'video')

            # 영상 다운로드
            ydl.download([url])

            # 다운로드된 파일 경로 찾기
            for file in os.listdir(output_path):
                if file.endswith(('.mp4', '.mkv', '.webm')):
                    video_path = os.path.join(output_path, file)
                    print(f"다운로드 완료: {video_path}")
                    return video_path

    except Exception as e:
        print(f"다운로드 오류: {e}")
        return None

# 영상 처리 함수
def process_video(video_path, output_path="./output_video.mp4"):
    """영상에서 객체 탐지를 수행하는 함수"""
    cap = cv2.VideoCapture(video_path)

    # 영상 속성 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"영상 정보: {width}x{height}, FPS: {fps}, 총 프레임: {total_frames}")

    # 비디오 작성기 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    # 프레임별 처리
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 객체 탐지 (기존 코드와 동일한 방식)
        results = model(frame)

        # 탐지 결과를 프레임에 그리기
        annotated_frame = results[0].plot()

        # 프레임 저장
        out.write(annotated_frame)

        # 진행률 표시
        if frame_count % 30 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"처리 진행률: {progress:.1f}%")

        frame_count += 1

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return output_path

# 방법 1: 유튜브 URL 직접 입력
youtube_url = input("유튜브 영상 URL을 입력하세요: ")

if youtube_url:
    # 유튜브 영상 다운로드
    video_path = download_youtube_video(youtube_url)

    if video_path:
        # 다운로드한 영상에 대해 YOLOv11n 모델로 추론 실행
        output_video = process_video(video_path)
    else:
        print("영상 다운로드에 실패했습니다.")

print("\n처리 완료!")

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

train: Scanning /content/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 922.9±214.9 MB/s, size: 54.0 KB)



val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

Plotting labels to runs/detect/train2/labels.jpg... 


...

                   all          4         17      0.652       0.85      0.861       0.63



10 epochs completed in 0.013 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


                   all          4         17      0.648       0.85      0.853      0.651
                person          3         10      0.652        0.6      0.599      0.284
                   dog          1          1      0.577          1      0.995      0.796
                 horse          1          2      0.752          1      0.995      0.674
              elephant          1          2      0.408        0.5      0.537      0.263
              umbrella          1          1      0.607          1      0.995      0.995
          potted plant          1          1      0.895          1      0.995      0.895
Speed: 2.3ms preprocess, 207.6ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train2
유튜브 영상 URL을 입력하세요: https://www.youtube.com/watch?v=KtBa71KDNkQ
[youtube] Extracting URL: https://www.youtube.com/watch?v=KtBa71KDNkQ
[youtube] KtBa71KDNkQ: Downloading webpage
[youtube] KtBa71KDNkQ: Downloading tv client config
[youtube] KtBa71KDNkQ: Downlo

error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
